In [38]:
import geopandas as gpd
import pandas as pd
import numpy as np

cleaned_buildings = gpd.read_file(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/buildings/cluster_with_buildings.geojson"
)

# Calcolo della densità abitativa su tutta la shape.
house = cleaned_buildings[cleaned_buildings["tags_building"] == "house"]
area_tot = house["area_m2"].sum()
pop_microgrid = (
    93641.59  # Qua devi usare la funzione nel codice per calcolare la popolazione.
)
density = pop_microgrid / area_tot

# Calcolo della house-area per ogni cluster.
grouped_buildings = cleaned_buildings.groupby("cluster_id")
clusters = np.sort(cleaned_buildings["cluster_id"].unique())
house_area_for_cluster = []
for cluster in clusters:
    cluster_buildings = pd.DataFrame(grouped_buildings.get_group(cluster))
    house = cluster_buildings[cluster_buildings["tags_building"] == "house"]
    area_house = house["area_m2"].sum()
    house_area_for_cluster.append(area_house)

# Creazione di un DataFrame con la popolazione suddivisa nei cluster sulla base della densità-area_house
population_df = pd.DataFrame()
population_df["cluster"] = clusters
population_df.set_index("cluster", inplace=True)
population_df["house_area_for_cluster"] = house_area_for_cluster
people_for_cluster = (population_df["house_area_for_cluster"] * density).round()
population_df["people_for_cluster"] = people_for_cluster

display(population_df)

,house_area_for_cluster,people_for_cluster
cluster,,
0,167199.174127,7850.0
1,198012.745308,9297.0
2,235799.960999,11071.0
3,233431.361591,10960.0
4,246815.535349,11588.0
5,252566.604783,11858.0
6,231201.805873,10855.0
7,60611.022440,2846.0
8,196700.620729,9235.0


In [39]:
# Creazione di un DataFrame con la popolazione per ogni cluster suddivisa nei vari Tier
tier_percent = [0.2, 0.2, 0.3, 0.2, 0.05, 0.05]
people_for_cluster = population_df["people_for_cluster"]
tier_pop_df = population_df["people_for_cluster"].apply(
    lambda x: pd.Series([x * y for y in tier_percent])
)

display(tier_pop_df)

,0,1,2,3,4,5
cluster,,,,,,
0,1570.0,1570.0,2355.0,1570.0,392.50,392.50
1,1859.4,1859.4,2789.1,1859.4,464.85,464.85
2,2214.2,2214.2,3321.3,2214.2,553.55,553.55
3,2192.0,2192.0,3288.0,2192.0,548.00,548.00
4,2317.6,2317.6,3476.4,2317.6,579.40,579.40
5,2371.6,2371.6,3557.4,2371.6,592.90,592.90
6,2171.0,2171.0,3256.5,2171.0,542.75,542.75
7,569.2,569.2,853.8,569.2,142.30,142.30
8,1847.0,1847.0,2770.5,1847.0,461.75,461.75


In [40]:
# Importazione dei profili medi e della deviazione standard per ogni Tier.
demand_tier_1 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier1.xlsx"
)
demand_tier_2 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier2.xlsx"
)
demand_tier_3 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier3.xlsx"
)
demand_tier_4 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier4.xlsx"
)
demand_tier_5 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier5.xlsx"
)

# Creazione di un DataFrame con tutti i tier e la domanda media oraria per ognuno
mean_demand_tier_df = pd.DataFrame()
demand_tiers = [
    demand_tier_1,
    demand_tier_2,
    demand_tier_3,
    demand_tier_4,
    demand_tier_5,
]

for i, demand_tier in enumerate(demand_tiers, start=1):
    mean_column_name = f"tier_{i}"
    mean_demand_tier_df[mean_column_name] = demand_tier["mean"]
mean_demand_tier_df.insert(0, "tier_0", np.zeros(len(mean_demand_tier_df)))

hours_index = pd.date_range(
    start="00:00:00", periods=len(mean_demand_tier_df), freq="H", normalize=True
)
mean_demand_tier_df.index = hours_index.time

display(mean_demand_tier_df)

C:\Users\margh\AppData\Local\Temp\ipykernel_21028\4291676238.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hours_index = pd.date_range(start='00:00:00', periods=len(mean_demand_tier_df), freq='H', normalize=True)


,tier_0,tier_1,tier_2,tier_3,tier_4,tier_5
00:00:00,0.0,0.000000,1.522227,1.196115,69.661717,1.462838
01:00:00,0.0,0.000000,4.056604,3.279885,83.855136,3.986523
02:00:00,0.0,0.000000,5.699268,4.795340,88.435496,5.483456
03:00:00,0.0,0.000000,9.040417,12.131634,98.316785,13.962898
04:00:00,0.0,0.000000,21.776059,40.681523,129.949732,47.042389
05:00:00,0.0,0.000000,35.400828,74.481385,168.448116,85.517763
06:00:00,0.0,0.249314,32.383003,64.871434,190.799572,91.267533
07:00:00,0.0,4.587977,23.531943,41.434136,217.681463,82.627530
08:00:00,0.0,0.364458,13.264789,23.872662,205.082100,51.110287
09:00:00,0.0,0.000000,5.452674,12.292701,185.331735,20.680141


In [41]:
# Passare dal DataFrame per la singola giornata a quello per tutto l'anno ( ripetendo il giorno tipo)

date_range = pd.date_range(start="2013-01-01", end="2013-12-31", freq="D")
yearly_mean_demand_tier_df = pd.concat(
    [mean_demand_tier_df] * len(date_range), ignore_index=True
)
date_time_index = pd.date_range(start="2013-01-01", end="2013-12-31 23:00:00", freq="H")
yearly_mean_demand_tier_df.index = date_time_index

display(yearly_mean_demand_tier_df)

C:\Users\margh\AppData\Local\Temp\ipykernel_21028\3427527519.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_time_index = pd.date_range(start='2013-01-01', end='2013-12-31 23:00:00', freq='H')


,tier_0,tier_1,tier_2,tier_3,tier_4,tier_5
2013-01-01 00:00:00,0.0,0.000000,1.522227,1.196115,69.661717,1.462838
2013-01-01 01:00:00,0.0,0.000000,4.056604,3.279885,83.855136,3.986523
2013-01-01 02:00:00,0.0,0.000000,5.699268,4.795340,88.435496,5.483456
2013-01-01 03:00:00,0.0,0.000000,9.040417,12.131634,98.316785,13.962898
2013-01-01 04:00:00,0.0,0.000000,21.776059,40.681523,129.949732,47.042389
...,...,...,...,...,...,...
2013-12-31 19:00:00,0.0,47.652813,62.252508,58.018541,472.705325,117.554121
2013-12-31 20:00:00,0.0,55.958416,76.411150,84.322244,321.723879,151.491828
2013-12-31 21:00:00,0.0,36.647671,67.905680,93.383237,249.302548,149.749051
2013-12-31 22:00:00,0.0,19.730346,43.333531,76.202498,193.773689,112.843233


In [43]:
# Creazione di un DataFrame per ogni Cluster contente la domanda media oraria per ogni tier ( intesa come domanda del singolo* n persone nel tier)
dataframes_dict = {}

for index, row in tier_pop_df.iterrows():
    loads = []
    for col in range(tier_pop_df.shape[1]):
        n = row[col]
        tier = yearly_mean_demand_tier_df.iloc[:, col]
        loads_h = n * tier
        loads.append(loads_h)

    loads_df = pd.DataFrame(loads)
    loads_df = loads_df.T

    nome_dataframe = f"loads_cluster_{index}"
    dataframes_dict[nome_dataframe] = loads_df

display(dataframes_dict["loads_cluster_0"])

,tier_0,tier_1,tier_2,tier_3,tier_4,tier_5
2013-01-01 00:00:00,0.0,0.000000,3584.845123,1877.900390,27342.224097,574.163991
2013-01-01 01:00:00,0.0,0.000000,9553.301569,5149.418798,32913.140761,1564.710097
2013-01-01 02:00:00,0.0,0.000000,13421.775775,7528.684329,34710.932302,2152.256445
2013-01-01 03:00:00,0.0,0.000000,21290.183211,19046.666059,38589.338168,5480.437642
2013-01-01 04:00:00,0.0,0.000000,51282.619050,63869.990653,51005.269774,18464.137717
...,...,...,...,...,...,...
2013-12-31 19:00:00,0.0,74814.916587,146604.657300,91089.109140,185536.839884,46139.992570
2013-12-31 20:00:00,0.0,87854.713886,179948.257914,132385.922447,126276.622590,59460.542445
2013-12-31 21:00:00,0.0,57536.842694,159917.876158,146611.682393,97851.250006,58776.502469
2013-12-31 22:00:00,0.0,30976.643920,102050.464746,119637.922365,76056.172739,44290.968758


In [44]:
tot_loads_dict = {}
for key in dataframes_dict:
    load = dataframes_dict[key]
    load_tot = load.sum(axis=1)

    total_loads_df = pd.DataFrame(load_tot, columns=[f"Total_{key}"])
    tot_loads_dict[key] = total_loads_df

display(tot_loads_dict["loads_cluster_0"])

,Total_loads_cluster_0
2013-01-01 00:00:00,33379.133600
2013-01-01 01:00:00,49180.571225
2013-01-01 02:00:00,57813.648851
2013-01-01 03:00:00,84406.625080
2013-01-01 04:00:00,184622.017194
...,...
2013-12-31 19:00:00,544185.515481
2013-12-31 20:00:00,585926.059282
2013-12-31 21:00:00,520694.153721
2013-12-31 22:00:00,373012.172528


In [45]:
tot_loads_df = pd.DataFrame()
for key, cluster_load in tot_loads_dict.items():
    tot_loads_df = pd.concat([tot_loads_df, cluster_load], axis=1)
tot_loads_df.to_csv(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/demand/microgrid_load_2.csv"
)

tot_loads_df

,Total_loads_cluster_0,Total_loads_cluster_1,Total_loads_cluster_2,Total_loads_cluster_3,Total_loads_cluster_4,Total_loads_cluster_5,Total_loads_cluster_6,Total_loads_cluster_7,Total_loads_cluster_8,Total_loads_cluster_9
2013-01-01 00:00:00,33379.133600,39531.949692,47075.208674,46603.223473,49273.554161,50421.626272,46156.750985,12101.530475,39268.318318,34365.625192
2013-01-01 01:00:00,49180.571225,58246.085437,69360.268030,68664.848488,72599.294186,74290.855234,68007.019191,17830.306459,57857.652900,50634.060719
2013-01-01 02:00:00,57813.648851,68470.508710,81535.656870,80718.164511,85343.256419,87331.751348,79944.860927,20960.209507,68013.891356,59522.281531
2013-01-01 03:00:00,84406.625080,99965.400429,119040.222453,117846.702022,124599.232029,127502.389834,116717.696209,30601.433755,99298.749377,86901.190305
2013-01-01 04:00:00,184622.017194,218653.617051,260375.841064,257765.262222,272535.023598,278885.080240,255295.795750,66934.300756,217195.455896,190078.362160
...,...,...,...,...,...,...,...,...,...,...
2013-12-31 19:00:00,544185.515481,644495.890118,767474.884318,759780.031806,803314.873044,822032.081857,752501.117267,197293.245485,640197.864391,560268.450461
2013-12-31 20:00:00,585926.059282,693930.518872,826342.344244,818057.275126,864931.359868,885084.230697,810220.047581,212426.186588,689302.822608,603242.600143
2013-12-31 21:00:00,520694.153721,616674.337216,734344.582910,726981.901246,768637.433544,786546.659213,720017.202375,188776.504648,612561.848358,536082.821704
2013-12-31 22:00:00,373012.172528,441769.957707,526065.956951,520791.517314,550632.491116,563462.209152,515802.182522,135234.731594,438823.874306,384036.226545


### REGOLA BUILD DEMAND CON DEVIAZIONE STANDARD

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

cleaned_buildings = gpd.read_file(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/buildings/cluster_with_buildings.geojson"
)

# Calcolo della densità abitativa su tutta la shape.
house = cleaned_buildings[cleaned_buildings["tags_building"] == "house"]
area_tot = house["area_m2"].sum()
pop_microgrid = (
    93641.59  # Qua devi usare la funzione nel codice per calcolare la popolazione.
)
density = pop_microgrid / area_tot

# Calcolo della house-area per ogni cluster.
grouped_buildings = cleaned_buildings.groupby("cluster_id")
clusters = np.sort(cleaned_buildings["cluster_id"].unique())
house_area_for_cluster = []
for cluster in clusters:
    cluster_buildings = pd.DataFrame(grouped_buildings.get_group(cluster))
    house = cluster_buildings[cluster_buildings["tags_building"] == "house"]
    area_house = house["area_m2"].sum()
    house_area_for_cluster.append(area_house)

# Creazione di un DataFrame con la popolazione suddivisa nei cluster sulla base della densità-area_house
population_df = pd.DataFrame()
population_df["cluster"] = clusters
population_df.set_index("cluster", inplace=True)
population_df["house_area_for_cluster"] = house_area_for_cluster
people_for_cluster = (population_df["house_area_for_cluster"] * density).round()
population_df["people_for_cluster"] = people_for_cluster

display(population_df)

,house_area_for_cluster,people_for_cluster
cluster,,
0,167199.174127,7850.0
1,198012.745308,9297.0
2,235799.960999,11071.0
3,233431.361591,10960.0
4,246815.535349,11588.0
5,252566.604783,11858.0
6,231201.805873,10855.0
7,60611.022440,2846.0
8,196700.620729,9235.0


In [2]:
# Creazione di un DataFrame con la popolazione per ogni cluster suddivisa nei vari Tier
tier_percent = [0.2, 0.2, 0.3, 0.2, 0.05, 0.05]
people_for_cluster = population_df["people_for_cluster"]
tier_pop_df = population_df["people_for_cluster"].apply(
    lambda x: pd.Series([x * y for y in tier_percent])
)

display(tier_pop_df)

,0,1,2,3,4,5
cluster,,,,,,
0,1570.0,1570.0,2355.0,1570.0,392.50,392.50
1,1859.4,1859.4,2789.1,1859.4,464.85,464.85
2,2214.2,2214.2,3321.3,2214.2,553.55,553.55
3,2192.0,2192.0,3288.0,2192.0,548.00,548.00
4,2317.6,2317.6,3476.4,2317.6,579.40,579.40
5,2371.6,2371.6,3557.4,2371.6,592.90,592.90
6,2171.0,2171.0,3256.5,2171.0,542.75,542.75
7,569.2,569.2,853.8,569.2,142.30,142.30
8,1847.0,1847.0,2770.5,1847.0,461.75,461.75


In [23]:
# Importazione dei profili medi e della deviazione standard per ogni Tier.
demand_tier_1 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier1.xlsx"
)
demand_tier_2 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier2.xlsx"
)
demand_tier_3 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier3.xlsx"
)
demand_tier_4 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier4.xlsx"
)
demand_tier_5 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier5.xlsx"
)

# Creazione di un DataFrame con tutti i tier e la domanda media oraria per ognuno
mean_demand_tier_df = pd.DataFrame()
demand_tiers = [
    demand_tier_1,
    demand_tier_2,
    demand_tier_3,
    demand_tier_4,
    demand_tier_5,
]

for i, demand_tier in enumerate(demand_tiers, start=1):
    mean_column_name = f"tier_{i}"
    mean_demand_tier_df[mean_column_name] = demand_tier["mean"]
mean_demand_tier_df.insert(0, "tier_0", np.zeros(len(mean_demand_tier_df)))

hours_index = pd.date_range(
    start="00:00:00", periods=len(mean_demand_tier_df), freq="H", normalize=True
)
mean_demand_tier_df.index = hours_index.time

# Creazione di un DataFrame con tutti i tier e la std media oraria per ognuno
std_demand_tier_df = pd.DataFrame()

for i, demand_tier in enumerate(demand_tiers, start=1):
    mean_column_name = f"tier_{i}"
    std_demand_tier_df[mean_column_name] = demand_tier["std"]
std_demand_tier_df.insert(0, "tier_0", np.zeros(len(mean_demand_tier_df)))

std_demand_tier_df.index = hours_index.time

display(mean_demand_tier_df)
display(std_demand_tier_df)

C:\Users\margh\AppData\Local\Temp\ipykernel_21028\610781103.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hours_index = pd.date_range(start='00:00:00', periods=len(mean_demand_tier_df), freq='H', normalize=True)


,tier_0,tier_1,tier_2,tier_3,tier_4,tier_5
00:00:00,0.0,0.000000,1.522227,1.196115,69.661717,1.462838
01:00:00,0.0,0.000000,4.056604,3.279885,83.855136,3.986523
02:00:00,0.0,0.000000,5.699268,4.795340,88.435496,5.483456
03:00:00,0.0,0.000000,9.040417,12.131634,98.316785,13.962898
04:00:00,0.0,0.000000,21.776059,40.681523,129.949732,47.042389
05:00:00,0.0,0.000000,35.400828,74.481385,168.448116,85.517763
06:00:00,0.0,0.249314,32.383003,64.871434,190.799572,91.267533
07:00:00,0.0,4.587977,23.531943,41.434136,217.681463,82.627530
08:00:00,0.0,0.364458,13.264789,23.872662,205.082100,51.110287
09:00:00,0.0,0.000000,5.452674,12.292701,185.331735,20.680141


,tier_0,tier_1,tier_2,tier_3,tier_4,tier_5
00:00:00,0.0,0.000000,0.429013,0.396138,2.178424,0.451707
01:00:00,0.0,0.000000,0.810468,0.709363,2.339555,0.766178
02:00:00,0.0,0.000000,0.942166,0.882295,2.547491,0.884457
03:00:00,0.0,0.000000,1.360764,2.102167,3.699718,2.631140
04:00:00,0.0,0.000000,2.758943,4.268914,6.274641,5.975277
05:00:00,0.0,0.000000,3.092914,4.510849,7.803500,6.827746
06:00:00,0.0,0.077539,3.229376,5.232349,8.991009,8.052097
07:00:00,0.0,0.366830,3.054168,3.960180,10.380958,7.643498
08:00:00,0.0,0.091126,2.232553,2.960420,9.327960,6.084947
09:00:00,0.0,0.000000,1.236567,2.187293,8.695498,3.763859


In [24]:
# Passare dal DataFrame per la singola giornata a quello per tutto l'anno ( ripetendo il giorno tipo)

date_range = pd.date_range(start="2013-01-01", end="2013-12-31", freq="D")
yearly_mean_demand_tier_df = pd.concat(
    [mean_demand_tier_df] * len(date_range), ignore_index=True
)
date_time_index = pd.date_range(start="2013-01-01", end="2013-12-31 23:00:00", freq="H")
yearly_mean_demand_tier_df.index = date_time_index

yearly_std_demand_tier_df = pd.concat(
    [std_demand_tier_df] * len(date_range), ignore_index=True
)
yearly_std_demand_tier_df.index = date_time_index

display(yearly_mean_demand_tier_df)
yearly_std_demand_tier_df

C:\Users\margh\AppData\Local\Temp\ipykernel_21028\1272881548.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_time_index = pd.date_range(start='2013-01-01', end='2013-12-31 23:00:00', freq='H')


,tier_0,tier_1,tier_2,tier_3,tier_4,tier_5
2013-01-01 00:00:00,0.0,0.000000,1.522227,1.196115,69.661717,1.462838
2013-01-01 01:00:00,0.0,0.000000,4.056604,3.279885,83.855136,3.986523
2013-01-01 02:00:00,0.0,0.000000,5.699268,4.795340,88.435496,5.483456
2013-01-01 03:00:00,0.0,0.000000,9.040417,12.131634,98.316785,13.962898
2013-01-01 04:00:00,0.0,0.000000,21.776059,40.681523,129.949732,47.042389
...,...,...,...,...,...,...
2013-12-31 19:00:00,0.0,47.652813,62.252508,58.018541,472.705325,117.554121
2013-12-31 20:00:00,0.0,55.958416,76.411150,84.322244,321.723879,151.491828
2013-12-31 21:00:00,0.0,36.647671,67.905680,93.383237,249.302548,149.749051
2013-12-31 22:00:00,0.0,19.730346,43.333531,76.202498,193.773689,112.843233


,tier_0,tier_1,tier_2,tier_3,tier_4,tier_5
2013-01-01 00:00:00,0.0,0.000000,0.429013,0.396138,2.178424,0.451707
2013-01-01 01:00:00,0.0,0.000000,0.810468,0.709363,2.339555,0.766178
2013-01-01 02:00:00,0.0,0.000000,0.942166,0.882295,2.547491,0.884457
2013-01-01 03:00:00,0.0,0.000000,1.360764,2.102167,3.699718,2.631140
2013-01-01 04:00:00,0.0,0.000000,2.758943,4.268914,6.274641,5.975277
...,...,...,...,...,...,...
2013-12-31 19:00:00,0.0,2.462683,4.666371,4.926717,23.364815,8.475493
2013-12-31 20:00:00,0.0,2.446845,4.953632,5.474969,14.945005,8.145099
2013-12-31 21:00:00,0.0,2.218095,4.743251,5.180570,9.373338,8.283509
2013-12-31 22:00:00,0.0,2.049108,4.280691,4.721086,7.820964,7.995370


In [25]:
# Creazione di un DataFrame per ogni Cluster contente la domanda media oraria per ogni tier ( intesa come domanda del singolo* n persone nel tier)
dataframes_dict = {}

for index, row in tier_pop_df.iterrows():
    loads = []
    for col in range(tier_pop_df.shape[1]):
        n = row[col]
        tier = yearly_mean_demand_tier_df.iloc[:, col]
        loads_h = n * tier
        loads.append(loads_h)

    loads_df = pd.DataFrame(loads)
    loads_df = loads_df.T

    nome_dataframe = f"loads_cluster_{index}"
    dataframes_dict[nome_dataframe] = loads_df


# Creazione di un DataFrame per ogni Cluster contente la std della domanda oraria per ogni tier ( intesa come random(std del singolo* sdrt(n persone nel tier)))

dataframes_std_dict = {}

for index, row in tier_pop_df.iterrows():
    loads = []
    for col in range(tier_pop_df.shape[1]):
        n = np.sqrt(row[col])
        tier = yearly_std_demand_tier_df.iloc[:, col]
        loads_h = n * tier
        loads.append(loads_h)

    loads_df = pd.DataFrame(loads)
    loads_df = loads_df.T
    loads_df = loads_df.applymap(lambda x: np.random.normal(0, x))

    nome_dataframe = f"std_cluster_{index}"
    dataframes_std_dict[nome_dataframe] = loads_df

display(dataframes_dict["loads_cluster_0"])
display(dataframes_std_dict["std_cluster_0"])

C:\Users\margh\AppData\Local\Temp\ipykernel_21028\4251276499.py:33: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  loads_df = loads_df.applymap(lambda x: np.random.normal(0, x))
C:\Users\margh\AppData\Local\Temp\ipykernel_21028\4251276499.py:33: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  loads_df = loads_df.applymap(lambda x: np.random.normal(0, x))
C:\Users\margh\AppData\Local\Temp\ipykernel_21028\4251276499.py:33: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  loads_df = loads_df.applymap(lambda x: np.random.normal(0, x))
C:\Users\margh\AppData\Local\Temp\ipykernel_21028\4251276499.py:33: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  loads_df = loads_df.applymap(lambda x: np.random.normal(0, x))
C:\Users\margh\AppData\Local\Temp\ipykernel_21028\4251276499.py:33: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map

,tier_0,tier_1,tier_2,tier_3,tier_4,tier_5
2013-01-01 00:00:00,0.0,0.000000,3584.845123,1877.900390,27342.224097,574.163991
2013-01-01 01:00:00,0.0,0.000000,9553.301569,5149.418798,32913.140761,1564.710097
2013-01-01 02:00:00,0.0,0.000000,13421.775775,7528.684329,34710.932302,2152.256445
2013-01-01 03:00:00,0.0,0.000000,21290.183211,19046.666059,38589.338168,5480.437642
2013-01-01 04:00:00,0.0,0.000000,51282.619050,63869.990653,51005.269774,18464.137717
...,...,...,...,...,...,...
2013-12-31 19:00:00,0.0,74814.916587,146604.657300,91089.109140,185536.839884,46139.992570
2013-12-31 20:00:00,0.0,87854.713886,179948.257914,132385.922447,126276.622590,59460.542445
2013-12-31 21:00:00,0.0,57536.842694,159917.876158,146611.682393,97851.250006,58776.502469
2013-12-31 22:00:00,0.0,30976.643920,102050.464746,119637.922365,76056.172739,44290.968758


,tier_0,tier_1,tier_2,tier_3,tier_4,tier_5
2013-01-01 00:00:00,0.0,0.000000,-37.829058,-14.368225,58.322886,4.783880
2013-01-01 01:00:00,0.0,0.000000,10.366171,11.675773,-23.643930,19.840311
2013-01-01 02:00:00,0.0,0.000000,27.592862,-20.786843,71.361285,-0.517295
2013-01-01 03:00:00,0.0,0.000000,63.468703,50.668832,124.265847,-46.026471
2013-01-01 04:00:00,0.0,0.000000,59.918252,152.466923,-278.281219,-159.718080
...,...,...,...,...,...,...
2013-12-31 19:00:00,0.0,-18.600262,-239.297599,339.258144,-200.159755,-244.514325
2013-12-31 20:00:00,0.0,289.170687,-332.609704,-27.041627,-86.291996,9.982130
2013-12-31 21:00:00,0.0,8.445888,66.954763,-140.169859,-50.234842,-162.867727
2013-12-31 22:00:00,0.0,-16.766311,-143.701441,-154.974268,-191.887086,68.225356


In [27]:
# Creazione di un dizionario dove salvare i dataframe con la somma del carico medio e della deviazione standard.

loads_with_std = {}

for i in range(len(dataframes_dict)):
    key_df = f"loads_cluster_{i}"
    key_std = f"std_cluster_{i}"

    # Somma i dataframe corrispondenti
    if key_df in dataframes_dict and key_std in dataframes_std_dict:
        df = dataframes_dict[key_df] + dataframes_std_dict[key_std]
        loads_with_std[key_df] = df
    else:
        print(f"Keys {key_df} o {key_std} not in dictionary.")

loads_with_std["loads_cluster_0"]

,tier_0,tier_1,tier_2,tier_3,tier_4,tier_5
2013-01-01 00:00:00,0.0,0.000000,3547.016065,1863.532165,27400.546983,578.947870
2013-01-01 01:00:00,0.0,0.000000,9563.667740,5161.094570,32889.496831,1584.550408
2013-01-01 02:00:00,0.0,0.000000,13449.368637,7507.897486,34782.293587,2151.739150
2013-01-01 03:00:00,0.0,0.000000,21353.651914,19097.334891,38713.604014,5434.411171
2013-01-01 04:00:00,0.0,0.000000,51342.537303,64022.457576,50726.988555,18304.419637
...,...,...,...,...,...,...
2013-12-31 19:00:00,0.0,74796.316325,146365.359701,91428.367284,185336.680129,45895.478244
2013-12-31 20:00:00,0.0,88143.884572,179615.648211,132358.880820,126190.330594,59470.524575
2013-12-31 21:00:00,0.0,57545.288582,159984.830921,146471.512534,97801.015164,58613.634742
2013-12-31 22:00:00,0.0,30959.877608,101906.763305,119482.948098,75864.285653,44359.194114


In [37]:
tot_loads_dict = {}
for key in loads_with_std:
    load = loads_with_std[key]
    load_tot = load.sum(axis=1)

    total_loads_df = pd.DataFrame(load_tot, columns=[f"Total_{key}"])
    tot_loads_dict[key] = total_loads_df

display(tot_loads_dict["loads_cluster_0"])


tot_loads_df = pd.DataFrame()
for key, cluster_load in tot_loads_dict.items():
    tot_loads_df = pd.concat([tot_loads_df, cluster_load], axis=1)
tot_loads_df.to_csv(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/demand/microgrid_load_3.csv"
)

tot_loads_df

,Total_loads_cluster_0
2013-01-01 00:00:00,33390.043084
2013-01-01 01:00:00,49198.809549
2013-01-01 02:00:00,57891.298861
2013-01-01 03:00:00,84599.001990
2013-01-01 04:00:00,184396.403070
...,...
2013-12-31 19:00:00,543822.201684
2013-12-31 20:00:00,585779.268772
2013-12-31 21:00:00,520416.281943
2013-12-31 22:00:00,372573.068778


,Total_loads_cluster_0,Total_loads_cluster_1,Total_loads_cluster_2,Total_loads_cluster_3,Total_loads_cluster_4,Total_loads_cluster_5,Total_loads_cluster_6,Total_loads_cluster_7,Total_loads_cluster_8,Total_loads_cluster_9
2013-01-01 00:00:00,33390.043084,39589.648516,47208.848589,46591.103226,49193.895086,50405.080128,46171.396492,12080.548982,39229.059418,34371.823529
2013-01-01 01:00:00,49198.809549,58231.258632,69337.468272,68664.725836,72631.350648,74205.262282,68127.495567,17850.721721,57864.371105,50610.040407
2013-01-01 02:00:00,57891.298861,68574.831767,81434.588995,80760.323301,85357.841080,87285.440382,79987.120329,20909.858498,68071.320558,59538.750698
2013-01-01 03:00:00,84599.001990,99780.340223,119039.824284,117882.494970,124378.880300,127341.525391,116640.363653,30614.543332,99394.646228,87033.116483
2013-01-01 04:00:00,184396.403070,218583.505108,260615.084705,258132.464530,272285.312853,278763.526783,255717.333814,66891.644142,217267.288947,190137.732519
...,...,...,...,...,...,...,...,...,...,...
2013-12-31 19:00:00,543822.201684,645218.519959,766059.156422,760802.091123,803946.531768,824021.898088,754087.259529,197541.743645,640153.053067,561113.267733
2013-12-31 20:00:00,585779.268772,694404.549964,826269.044290,817965.021305,865382.100877,883617.050366,810067.271565,212295.822012,689331.979339,602470.996323
2013-12-31 21:00:00,520416.281943,616515.153839,733576.489297,727257.081579,768270.345764,785796.456386,720510.173589,188752.092563,612487.235326,535997.864508
2013-12-31 22:00:00,372573.068778,442405.246851,525887.080315,520850.856570,550040.548132,562611.685256,516221.157525,135305.883004,438939.128646,384440.613687
